# Automation Scripts 

## General Initilzation

In [ ]:
import time
from collections import OrderedDict
import numpy as np
import nidaqmx
import pandas as pd
from scipy import signal
import random

import quantities as pq
from nidaqmx.stream_writers import (
    DigitalSingleChannelWriter, DigitalMultiChannelWriter)
from nidaqmx.utils import flatten_channel_string
from nidaqmx.constants import (
    LineGrouping, AcquisitionType, DigitalWidthUnits, Edge,
    HandshakeStartCondition, Level, MIOAIConvertTimebaseSource,
    OverflowBehavior, TaskMode, Polarity, RegenerationMode,
    SampleInputDataWhen, SampleTimingType, UnderflowBehavior)
from nidaqmx.error_codes import DAQmxErrors, DAQmxWarnings
from nidaqmx.errors import (
    check_for_error, is_string_buffer_too_small, DaqError, DaqResourceWarning)

from scipy.optimize import fsolve, least_squares

from olfactometer.PID_reader import PID_Tester
from olfactometer.valve_driver import ValveDriver
from olfactometer.equipment import Olfactometer
from olfactometer.my_equipment import MyValve, MyJar, MyLowMFC, \
                                      MyMediumMFC, MyHighMFC
from olfactometer.odorants import Solution, Compound, ChemicalOrder, \
                                  Vendor, Molecule
from olfactometer.smell_engine import SmellEngine
# from graph import make_graph, draw_graph
from pprint import pprint
from olfactometer.smell_engine_communicator import SmellEngineCommunicator
from olfactometer.data_container import DataContainer
from olfactometer.ui import UI
from IPython.display import display

np.set_printoptions(precision=6)

In [ ]:
NUM_ODORANTS = 1

In [ ]:
molecules = OrderedDict([(702, 'Ethanol')])
# Initialize UI
ui = UI(molecules)
NUM_PID_SAMPLES = 10000

In [ ]:
eth = "./ethanol_11-2-21.pkl"

In [ ]:
smell_engine = SmellEngine(250, len(molecules),DataContainer(), debug_mode=False, write_flag=False, PID_mode = True,look_up_table_path=eth, oms=molecules)
# Initialize system
smell_engine.set_odorant_molecule_ids(list(molecules.keys()))
smell_engine.set_odorant_molecule_dilutions([1])
smell_engine.initialize_smell_engine_system()
smell_engine.smell_controller.valve_driver.num_pid_samples = NUM_PID_SAMPLES
smell_engine.olfactometer.loaded_molecules

In [ ]:
# Tell me how to get max_flow_rates and n_jars from smell_engine
n_jars = len(smell_engine.olfactometer.jars)
print(n_jars)
max_flow_rates = smell_engine.smell_controller.get_max_flow_rates()
total_vapor = smell_engine.smell_controller.get_vapor_concs_dense(set(list(smell_engine.target_concentration))).sum(axis=0)
print(f"Max Flow Rate {max_flow_rates}\nVapor Pressures {total_vapor}")

In [ ]:
# Timer setup specifies sampling frequency, sampling rate specifics # of samples to read
PID_mode = True
pid = PID_Tester(ui, smell_engine, PID_mode, cont_read_conc=False,sampling_rate = 10000)

pid.timer_setup(.00001)
pid.timer_start()
#display(ui.timeSeries)

In [ ]:
len(ui.timeSeriesData)

Automation Grapher

## Concentration Automation

In [ ]:
# FULL
concs = [0,0] + np.geomspace(1e-9,1e-3,49).tolist()
concs = concs[0:2] + concs[8:]
print(concs)

In [ ]:
# LOWER
concs = [0,0] + np.geomspace(1e-9,1e-3,49).tolist()
#concs = concs[0:41]
concs = concs[0:2] + concs[10:41]
print(concs)
print(len(concs))

In [ ]:
# HIGHER
concs = [0,0] + np.geomspace(1e-9,1e-3,49).tolist()
concs = [0,0] + concs[-25:]
print(concs)

In [ ]:
# Data Format
auto_data = {
    "info":"Ethanol 100% 11-3-2021 - Lower Concentrations, .25L constant flow",
    "data":[]
}

print(concs)

In [ ]:
len(auto_data["data"])

### Steady State

In [ ]:
#Steady State
smell_engine.set_desired_concentrations(NUM_ODORANTS*[0])

In [ ]:
import threading
import json
count_global = 0
baseline = 0 
init = True 
WAVE_ACCURACY = 500


def auto_collect():
    #global variables and variable declarations 
    global count_global
    global concs
    global baseline
    global init 
    l_concs = len(concs)
    
    print("COUNT" + str(count_global)+ " " + str(count_global%l_concs))
    print("CONC" + str(concs[count_global%l_concs]))
    
    threading.Timer(10, auto_collect).start()
        
    pid_average = sum(ui.timeSeries.data[0].y) /len(ui.timeSeries.data[0].y) #calculate the pid_average
    
    #establish condition to establish a new baseline value
    if(count_global%l_concs == 1):
        print(f"NEW BASELINE {pid_average}")
        baseline = pid_average
        pid_average_C = pid_average
    else: 
        pid_average_C = pid_average - baseline 
        
    #if not steady stater record data
    if(count_global%l_concs != 0):
        auto_data["data"].append({
            "conc":concs[count_global%l_concs],
            "pidC":pid_average_C,
            "pid":pid_average,
            "wave":signal.resample(ui.timeSeries.data[0].y, WAVE_ACCURACY).tolist()
        })
        
    #write data
    with open(f"auto_results_conc.json","w") as outfile:
            json.dump(auto_data,outfile)
    count_global = count_global + 1
    smell_engine.set_desired_concentrations([concs[count_global%l_concs]])
    print("END")
        
auto_collect()

In [ ]:
import json
with open(f"auto_results_conc_lower_11-3_250mL_const.json","w") as outfile:
        json.dump(auto_data,outfile)

## Machine State Automation

In [ ]:
# HIGH
# total_flow = 1000
# LOW
total_flow = 500

smell_engine.set_olfactometer_target_outflow(total_flow)

def set_machine(state_array):
    print([state_array[1],state_array[0],1-(state_array[1]+state_array[0])])
    print(state_array[2:22])
    valve_duty_cycle = [[i+1,0,0] for i in range(0,10)]

    for i in range(0,10):
        valve_duty_cycle[i][1] = state_array[2:22][i]
        valve_duty_cycle[i][2] = state_array[12:22][i]
    smell_engine.automation_set_mfc_setpoints([state_array[1],state_array[0],42])
    smell_engine.set_valve_duty_cycles(valve_duty_cycle)


In [ ]:
set_machine([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
#define data Frame of machine states to iterate through.

steady_state = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


states = [
    steady_state,
    steady_state,

         ]

In [ ]:
vvoA = []
vfoA = []
vvoB = []
vfoB = []
for v_flow in range(1,5):
    for v_duty in range(1,11):
        vvoA.append([v_flow/4, 0, v_duty/10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
vvoA.reverse()
random.shuffle(vvoA)
    
for v_duty in range(1,5):
    for v_flow in range(1,11):
        vfoA.append([v_flow/10, 0, v_duty/4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
vfoA.reverse()
random.shuffle(vfoA)
        
for v_flow in range(1,5):
    for v_duty in range(1,11):
        vvoB.append([0, v_flow/4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, v_duty/10, 0, 0, 0, 0, 0, 0, 0, 0, 0])
vvoB.reverse()
random.shuffle(vvoB)
    
for v_duty in range(1,5):
    for v_flow in range(1,11):
        vfoB.append([0, v_flow/10,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  v_duty/4, 0, 0, 0, 0, 0, 0, 0, 0, 0])
vfoB.reverse()
random.shuffle(vfoB)

In [ ]:
# all_states = vvoA + [steady_state]+ vfoA + [steady_state]+ vvoB + [steady_state]+  vfoB
all_states = vvoB + [steady_state]+  vfoB
states = states + all_states

In [ ]:
# Time estimate
((len(all_states)*3 * 30) / 60 )/60

In [ ]:
df_states = pd.DataFrame(states, columns=["mfcA","mfcB","h1","h2","h3","h4","h5","h6","h7","h8","h9","h10","L1","L2","L3","L4","L5","L6","L7","L8","L9","L10"])

In [ ]:
df_states

In [ ]:
import threading
import json
count_global = 0
baseline = 0 
init = True 
WAVE_ACCURACY = 500

print("starting loop")

def auto_collect():
    #global variables and variable declarations 
    l_concs = len(states)
    global df_states
    global baseline
    global init 
    global count_global
    global WAVE_ACCURACY
    
    print("COUNT" + str(count_global)+ " " + str(df_states.values[count_global%l_concs].tolist()))
    print("CONC" + str(df_states.values[count_global%l_concs].tolist()))
    
    #decide if to wait 5 minutes or 1 minute depending on if the steady state is resetting
    threading.Timer(10, auto_collect).start()
        
    pid_average = sum(ui.timeSeries.data[0].y) /len(ui.timeSeries.data[0].y) #calculate the pid_average
    
    #establish condition to establish a new baseline value
    if(count_global%l_concs == 1):
        print(f"NEW BASELINE {pid_average}")
        baseline = pid_average
        pid_average_C = pid_average
    else: 
        pid_average_C = pid_average - baseline 
        
    #print("SAMPLED")
    #print(signal.resample(ui.timeSeries.data[0].y, WAVE_ACCURACY).tolist())
    #if not steady stater record data
    if(count_global%l_concs != 0):
        auto_data["data"].append({
            "state":df_states.values[count_global%l_concs].tolist(),
            "pidC":pid_average_C,
            "pid":pid_average,
            "wave":signal.resample(ui.timeSeries.data[0].y, WAVE_ACCURACY).tolist()
        })
        
    #write data
    with open(f"auto_results_state.json","w") as outfile:
            json.dump(auto_data,outfile)
            
    count_global = count_global + 1
    set_machine(df_states.values[count_global%l_concs].tolist())
    print("END")
        
auto_collect()

In [ ]:
import json
with open(f"auto_results_11-1-mfcB_trials_mfcC_0p5L.json","w") as outfile:
        json.dump(auto_data,outfile)

In [ ]:
from scipy import signal
y = 1000* [1]

In [ ]:
len(y)

In [ ]:
downsampled =  signal.resample(y, 10).tolist()

In [ ]:
downsampled

In [ ]:
plt.plot(ui.timeSeriesData)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(signal.resample(ui.timeSeriesData,500))